#  Multilayer Perceptron Classification

<p style='text-align: justify;'> 
In this section we will study the concepts behind the classification problem in a MLP using Backpropagation.
</p>

## Objectives

* **Understand** how classification works in a MLP;
* **Learn** how to classify images with a MLP;
* **Explore** the different ways to classify images;
* **Create** classification MLP with backpropagation;
* **Train** the Multilayer Perceptron Network using Backpropagation to classify images;
* **Observe** how the MLP classify the images.

## The Problem: Classifying images in the Multilayer Perceptron

<p style='text-align: justify;'> 
Classifying images using an MLP is extremely common across various technology fields nowadays. However, to achieve high accuracy, it is necessary to implement image classification using backpropagation algorithms in an MLP (Multilayer Perceptron).
</p>  

## The Solution: Backpropagation

<p style='text-align: justify;'> 
Backpropagation is essential in MLPs for efficient error propagation, weight adjustment, and learning of non-linear relationships. It allows the network to iteratively update the weights based on gradients, resulting in improved performance and generalization to unseen data.
</p>  

## How Backpropagation works in a Neural Network

<p style='text-align: justify;'>
The Backpropagation is based on gradient descent, which aims to minimize the error between the network's predicted output and the target output. The backpropagation algorithm efficiently computes the gradient of the error function for the network's weights, allowing for iterative adjustment of the weights to minimize the error.

The key idea behind Backpropagation is to propagate the error from the output of the network layer back to its input layer. This process involves two main steps: forward propagation and backward propagation.
    
1. <b>Forward Propagation:</b>
During forward propagation, the input data is fed into the network, and the activations of each neuron are computed layer by layer until reaching the output layer. The activations are obtained by applying an activation function (such as sigmoid or ReLU) to each neuron's weighted sum of inputs.
    
2. <b>Backward Propagation:</b>
Once the output activations of a layer are computed, the error between the predicted output and the target output is calculated using an error function (e.g., mean squared error or cross-entropy). Then, the algorithm propagates this error back through the network to update the weights.
</p>

## Updating Weights and Bias in the Neural Network

<p style='text-align: justify;'>
In a neural network, the weights and biases are updated during training to optimize the network's performance. The backpropagation algorithm is commonly used to calculate the gradients of the weights and biases, which are then used to update their values. 

The main steps involved in the backward propagation are as follows:

1.  <b>Error Calculation:</b>
    The error is calculated by taking the derivative of the error function for the output layer's activations. This derivative represents how much each output activation contributes to the overall error.
    
2.  <b>Gradient Calculation:</b>
    The derivative of the error function for the weights of each neuron is computed. This is done by applying the chain rule, which allows the calculation of the result of a composite function. The gradient is calculated for each layer, starting from the output layer and moving backward. 

3.  <b>Weight Update:</b>
    The calculated gradients are then used to update the weights of the network. The weights are adjusted in the direction that minimizes the error based on the computed gradients and a learning rate parameter. The learning rate determines the step size taken during weight updates. 
    
4.  <b>Repeat:</b>
    Steps 2a to 2c are repeated for multiple iterations or until a stopping criterion is met (e.g., a predefined maximum number of iterations or reaching a satisfactory error level).

The backpropagation algorithm enables the network to learn the appropriate weights for making accurate predictions by iteratively adjusting the weights through forward and backward propagation. Updating the weights based on error gradients allows the network to improve its performance gradually.
</p>

## ☆ Challenger: Image Classification ☆

In this exercise, you will train an MLP to sort handwritten number images. The dataset used is [_MNIST_](https://en.wikipedia.org/wiki/MNIST_database), which consists of 16x16 pixels grayscale images of handwritten digits. Each image is represented as a vector of 256-pixel values, and each image is labeled with the digit it means (0 to 9). You must use an input layer with 256 neurons (to represent the image pixels), at least one hidden layer with the number you choose, and an output layer with ten neurons (one for each digit). Your goal is to train an MLP to sort these numbers correctly. Use the sigmoid activation function on all layers.

<div style="text-align:center">
<img src="./images/mnist1.png" style="width: 600px;">
</div>

### ⊗ Import Python Packages 

Let is start by importing all the modules we will need.

In [ ]:
import torch                       # for general PyTorch functionality
import torch.nn as nn              # for functional for neural network based functions
import torch.nn.functional as F    # for functional for neural network based functions
import torch.optim as optim        # for our optimizer which will update the parameters of our neural network
import torch.utils.data as data    # for handling the dataset

import torchvision.transforms as transforms # for data augmentation
import torchvision.datasets as datasets     # for loading the dataset

from sklearn import metrics                 # for visualizing a confusion matrix
from sklearn import decomposition           # for visualizing the neural network's representations in two dimensions
from sklearn import manifold                # for visualizing the neural network's representations in two dimensions  
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt             # for plotting

import numpy as np
import copy
import random
import time

The main ones we need to import are:

- The **torch** module is part of [_PyTorch_](https://pytorch.org), a popular machine learning library in Python.

- The **torchvision** is a package that provides access to popular datasets, model architectures, and image transformations for computer vision. It is generally used alongside PyTorch.

- The **sklearn** module is part of the [_Scikit-learn_](https://scikit-learn.org/stable/) package that is a powerful and widely used library in Python for machine learning. It provides a range of supervised and unsupervised learning algorithms in Python.


To ensure we get reproducible results we set the random seed for Python, Numpy and PyTorch.

In [ ]:
SEED = 12345

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### ⊗ Load the Dataset

The first thing we will do is load our dataset. This will automatically download the training set for the [_MNIST dataset_](https://en.wikipedia.org/wiki/MNIST_database).  This is a dataset of 28x28 black and white images consisting of handwritten digits, 0 to 9.  We will save it in a folder called `.data`. It will create the folder if it does not exist.

<div style="text-align:center">
<img src="./images/mlp-mnist.png" style="width: 500px;">
</div>

In [ ]:
ROOT = '.data'

train_data = datasets.MNIST(root=ROOT,
                            train=True,
                            download=True)

### ⊗ Normalize the Dataset

<p style='text-align: justify;'> 
Next, we want to normalize our data. This means we want it to have a mean of zero and a standard deviation of one.  We normalize our data by subtracting the mean and dividing by the standard deviation of our dataset. First, we need to calculate the mean and standard deviation.  To calculate the means and standard deviations we get the actual data (the images) using the `.data.` attribute of our training data, convert them into floating point numbers, and then use the built-in `mean` and `std` functions to calculate the mean and standard deviation, respectively. The image data has values between 0-255, which we want to scale between 0-1, so we divide by 255.
</p>

In [ ]:
mean = train_data.data.float().mean() / 255
std = train_data.data.float().std() / 255

In [ ]:
print(f'Value of calculated mean: {mean}')
print(f'Value of calculated std: {std}')

### ⊗ Data augmentation 

After calculated our mean and standard deviation we use Torchvision's `transforms`.  A `transform` states how our data should be augmented and processed. Data augmentation involves manipulating the available training data in a way that artificially creates more training examples. We use `transforms.Compose` to built a list of transformations that will be applied to the image. 

The transforms we use are:
- `RandomRotation` - randomly rotates the image between `(-x, +x)` degrees, where we have set `x = 5`. Note, the `fill=(0,)` is due to a [bug](https://github.com/pytorch/vision/issues/1759) in some versions of torchvision. 

- `RandomCrop` - this first adds `padding` around our image, 2 pixels here, to artificially make it bigger, before taking a random `28x28` square crop of the image.

- `ToTensor()` - this converts the image from a PIL image into a PyTorch tensor.

- `Normalize` - this subtracts the mean and divides by the standard deviations given. 

In [ ]:
train_transforms = transforms.Compose([
                                       transforms.RandomRotation(5, fill=(0,)),
                                       transforms.RandomCrop(28, padding=2),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[mean], std=[std])
                                      ])

test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[mean], std=[std])
                                     ])

### ⊗ Train and Test Data

Now we have defined our transforms we can then load the train and test data with the relevant transforms defined.

In [ ]:
train_data = datasets.MNIST(root=ROOT,
                            train=True,
                            download=True,
                            transform=train_transforms)

test_data = datasets.MNIST(root=ROOT,
                           train=False,
                           download=True,
                           transform=test_transforms)

We can simply check the `len` of the datasets to see how many examples are within each.

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

### ⊗ Plot Images

We can look at some images within our dataset to see what we're working with. The function below plots a square grid of images. If you supply less than a square number of images, it will ignore the last few. We will load 25 images, which have been processed through our transforms so that they will be randomly rotated and cropped.

In [ ]:
def plot_images(images):

    n_images = len(images)

    rows = int(np.sqrt(n_images))
    cols = int(np.sqrt(n_images))

    fig = plt.figure()
    for i in range(rows*cols):
        ax = fig.add_subplot(rows, cols, i+1)
        ax.imshow(images[i].view(28, 28).cpu().numpy(), cmap='bone')
        ax.axis('off')
        
N_IMAGES = 25

images = [image for image, label in [train_data[i] for i in range(N_IMAGES)]]

plot_images(images)    

### ⊗ Validation Data

The MNIST dataset comes with a training and test set, but not a validation set. We want to use a validation set to check how well our model performs on unseen data. Furthermore, we create a validation set, taking 10% of the training set. First, we have to define the exact number of examples that we want to be in each split of the training/validation sets.

In [ ]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

Then, we use the `random_split` function to take a random 10% of the training set to use as a validation set. The remaining 90% will stay as the training set.

In [ ]:
train_data, valid_data = data.random_split(train_data,
                                           [n_train_examples, n_valid_examples])

We can print out the number of examples again to check our splits are correct.

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

One thing to consider is that as the validation set has been created from the training set it has the same transforms as the training set, with the random rotating and cropping. As we want our validation set to act as a proxy for the test set, it should also be fixed, without any random augmentation.  First, let is see what 25 of the images within the validation set look like with the training transforms:

In [ ]:
N_IMAGES = 25

images = [image for image, label in [valid_data[i] for i in range(N_IMAGES)]]

plot_images(images)

We can now simply replace the validation set's transform by overwriting it with our test transforms from above.

As the validation set is a `Subset` of the training set, if we change the transforms of one, then by default Torchvision will change the transforms of the other. To stop this from happening, we make a `deepcopy` of the validation data.

In [ ]:
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

To double check we've correctly replaced the training transforms, we can view the same set of images and notice how they're more central (no random cropping) and have a more standard orientation (no random rotations).

In [ ]:
N_IMAGES = 25

images = [image for image, label in [valid_data[i] for i in range(N_IMAGES)]]

plot_images(images)

Next, we will define a `DataLoader` for each training/validation/test set. We can iterate over these, and they will yield batches of images and labels that we can use to train our model. We only need to shuffle our training set as it will be used for stochastic gradient descent, and we want each batch to be different between epochs. We aren't using the validation or test sets to update our model parameters, so they do not need to be shuffled. We want to use the most considerable batch size that we can. The 64 is relatively small and can be increased if our hardware can handle it.

In [ ]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

### ⊗ Defining the Model

Our model will be a neural network, specifically a multilayer perceptron (MLP) with two hidden layers. The image below shows the archicture of the model. 

<div style="text-align:center">
<img src="./images/mlp-mnist.png" style="width: 500px;">
</div>


Specifically, first we will flatten our 1x28x28 (1 color channel, 28 pixels height and width) image into a 784 element vector, also called 784 *features*. We flatten our input, as MLPs cannot handle two or three-dimensional data. Next, the 784 dimensional input is passed through the first hidden layer to transform it into 250 dimensions. Then, another hidden layer, which will transform it to 100 dimensions. Finally, an output layer which will transform it into a 10 dimensional vector. The output dimension should equal the number of classes within your data. Here we have ten digits, 0 - 9, so need our output to be 10 dimensions.

The transformation between 784 to 250, 250 to 100 and 100 to 10 dimensions are done by `Linear` layers. These are also known as fully connected or affine layers. In these layers, every element in one layer is connected to every element in the next. We can think of these elements as *neurons*, as this architecture is inspired by how the human brain is made of millions of interconnected nodes, also called neurons. 

Each connection between a neuron in one layer and a neuron in the next has a *weight* associated with it. The input to one neuron is the sum of the weighted values of all neurons in the previous layer connected to it, plus a weighted bias term, where the bias value is always 1. The neuron then applies an *activation function* to this weighted sum. This activation function is a non-linear function that allows the neural network to learn non-linear functions between inputs and outputs. 

We define our MLP below, which consists of three linear layers. We first take the input batch of images and flatten them, so they can be passed into the linear layers. We then pass them through the first linear layer, `input_fc`, which calculates the weighted sum of the inputs, and then apply the *ReLU* (rectified linear unit) activation function elementwise. This result is then passed through another linear layer, `hidden_fc`, again applying the same activation function elementwise. Finally, we pass this through the final linear layer, `output_fc`. We return not only the output but also the second hidden layer as we will do some analysis on it later.

### ⊗ Activation Function

The **ReLU activation function** is a popular non-linear function that is simply $max(0, x)$, where $x$ is the weighted sum of the inputs to that neuron. Other activation functions used are hyperbolic tan (tanh) and sigmoid function, however ReLU is the most commonly used.

One thing to note is that we do not use an activation function on the input directly or on the output. You should never use activation functions directly on the input, i.e. `F.relu(x)`. PyTorch combines activation functions to be applied on the output with the functions which calculate the *loss*, also known as *error* or *cost*, of a neural network. This is done for numerical stability.

In [ ]:
class MLP(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 250)
        self.hidden_fc = nn.Linear(250, 100)
        self.output_fc = nn.Linear(100, output_dim)

    def forward(self, x):
        batch_size = x.shape[0]

        x = x.view(batch_size, -1)

        h_1 = F.relu(self.input_fc(x))

        h_2 = F.relu(self.hidden_fc(h_1))

        y_pred = self.output_fc(h_2)

        return y_pred, h_2

We will define our model by creating an instance of it and setting the correct input and output dimensions.

In [ ]:
INPUT_DIM = 28 * 28
OUTPUT_DIM = 10

model = MLP(INPUT_DIM, OUTPUT_DIM)

We can also create a small function to calculate the number of trainable parameters (weights and biases) in our model - in case all of our parameters are trainable.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

The first layer has 784 neurons connected to 250 neurons, so 784*250 weighted connections plus 250 bias terms.

The second layer has 250 neurons connected to 100 neurons, 250*100 weighted connections plus 100 bias terms.

The third layer has 100 neurons connected to 10 neurons, 100*10 weighted connections plus 10 bias terms.

$$784 \cdot 250 + 250 + 250 \cdot 100 + 100 + 100 \cdot 10 + 10 = 222,360$$

In [ ]:
print(f'The model has {count_parameters(model):,} trainable parameters')

### ⊗ Training the Model

Next, we will define our optimizer. This is the algorithm we will use to update the parameters of our model with respect to the loss calculated on the data:

- Pass a batch of data through your model,

- Calculate the loss of your batch by comparing your predictions of model against the actual labels,

- Calculate the gradient of each of your parameters with respect to the loss,

- Update each of your parameters by subtracting their gradient multiplied by a small *learning rate* parameter.

We use the *Adam* algorithm with the default parameters to update our model. Improved results could be obtained by searching over different optimizers and learning rates, however default Adam is usually a good starting off point. 

In [ ]:
optimizer = optim.Adam(model.parameters())

Then, we define a *criterion*, PyTorch's name for a loss/cost/error function. This function will take in your model's predictions with the actual labels and then compute the loss/cost/error of your model with its current parameters.

`CrossEntropyLoss` both computes the *softmax* activation function on the supplied predictions as well as the actual loss via *negative log likelihood*. 

Briefly, the softmax function is:

$$\text{softmax }(\mathbf{x}) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$ 

This turns out 10 dimensional output, where each element is an unbounded real number, into a probability distribution over 10 elements. That is, all values are between 0 and 1, and together they all sum to 1. 

So we can use negative log likelihood for our loss function, as it expects probabilities. PyTorch calculates negative log likelihood for a single example via:

$$\text{negative log likelihood }(\mathbf{\hat{y}}, y) = -\log \big( \text{softmax}(\mathbf{\hat{y}})[y] \big)$$

$\mathbf{\hat{y}}$ is the $\mathbb{R}^{10}$ output, from our neural network, whereas $y$ is the label, an integer representing the class. The loss is the negative log of the class index of the softmax. For example:

$$\mathbf{\hat{y}} = [5,1,1,1,1,1,1,1,1,1]$$

$$\text{softmax }(\mathbf{\hat{y}}) = [0.8585, 0.0157, 0.0157, 0.0157, 0.0157, 0.0157, 0.0157, 0.0157, 0.0157, 0.0157]$$

If the label was class zero, the loss would be:

$$\text{negative log likelihood }(\mathbf{\hat{y}}, 0) = - \log(0.8585) = 0.153 \dots$$

If the label was class five, the loss would be:

$$\text{negative log likelihood }(\mathbf{\hat{y}}, 5) = - \log(0.0157) = 4.154 \dots$$

So, intuitively, as your model's output corresponding to the correct class index increases, your loss decreases.

In [ ]:
criterion = nn.CrossEntropyLoss()

We then define `device`. This is used to place your model and data on to a GPU, if you have one.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We place our model and criterion on to the device by using the `.to` method.

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

Next, we'll define a function to calculate the accuracy of our model. This takes the index of the highest value for your prediction and compares it against the actual class label. We then divide how many our model got correct by the amount in the batch to calculate accuracy across the batch.

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

We finally define our training loop.This will:

- Put our model into `train` mode,
- Iterate over our dataloader, returning batches of (image, label),
- Place the batch on to our GPU, if we have one,
- Clear the gradients calculated from the last batch,
- Pass our batch of images, `x`, through to model to get predictions, `y_pred`,
- Calculate the loss between our predictions and the actual labels,
- Calculate the accuracy between our predictions and the actual labels,
- Calculate the gradients of each parameter,
- Update the parameters by taking an optimizer step,
- Update our metrics.

Some layers act differently when training and evaluating the model that contains them, hence why we must tell our model we are in "training" mode. The model we are using here does not use any of those layers, however it is good practice to get used to putting your model in training mode.

In [ ]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The evaluation loop is similar to the training loop. The differences are:
- we put our model into evaluation mode with `model.eval()`
- we wrap the iterations inside a `with torch.no_grad()`
- we do not zero gradients as we are not calculating any
- we do not calculate gradients as we are not updating parameters
- we do not take an optimizer step as we are not calculating gradients

`torch.no_grad()` ensures that gradients are not calculated for whatever is inside the `with` block. As our model will not have to calculate gradients, it will be faster and use less memory. 

In [ ]:
def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The final step before training is to define a small function to tell us how long an epoch took.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We are finally ready to train!

During each epoch we calculate the training loss and accuracy, followed by the validation loss and accuracy. We then check if the validation loss achieved is the best validation loss we have seen. If so, we save our model's parameters (called a `state_dict`).

In [ ]:
EPOCHS = 10

best_valid_loss = float('inf')

for epoch in trange(EPOCHS):

    start_time = time.monotonic()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Afterwards, we load our the parameters of the model that achieved the best validation loss and then use this to evaluate our model on the test set.

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

Our model achieves 98% accuracy on the test set.

This can be improved by tweaking hyperparameters, e.g. number of layers, number of neurons per layer, optimization algorithm used, learning rate, etc. 

In [ ]:
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

### ⊗ Examining the Model

Now we have trained our model, there are a few things we can look at. Most of these are simple exploratory analysis, but they can offer some insights into your model. An important thing to do is check what examples your model gets wrong and ensure that they're reasonable mistakes.

The function below will return the predictions of model over a given dataset. It will return the inputs (image) the outputs (model predictions) and the ground truth labels.

In [ ]:
def get_predictions(model, iterator, device):

    model.eval()

    images = []
    labels = []
    probs = []

    with torch.no_grad():

        for (x, y) in iterator:

            x = x.to(device)

            y_pred, _ = model(x)

            y_prob = F.softmax(y_pred, dim=-1)

            images.append(x.cpu())
            labels.append(y.cpu())
            probs.append(y_prob.cpu())

    images = torch.cat(images, dim=0)
    labels = torch.cat(labels, dim=0)
    probs = torch.cat(probs, dim=0)

    return images, labels, probs

We can then get these predictions and, by taking the index of the highest predicted probability, get the predicted labels.

In [ ]:
images, labels, probs = get_predictions(model, test_iterator, device)

pred_labels = torch.argmax(probs, 1)

Then, we can make a confusion matrix from our actual labels and our predicted labels.

In [ ]:
def plot_confusion_matrix(labels, pred_labels):

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    cm = metrics.confusion_matrix(labels, pred_labels)
    cm = metrics.ConfusionMatrixDisplay(cm, display_labels=range(10))
    cm.plot(values_format='d', cmap='Blues', ax=ax)

The results seem reasonable enough, the most confused predictions-actuals are: 3-5 and 2-7.

In [ ]:
plot_confusion_matrix(labels, pred_labels)

Next, for each of our examples, we can check if our predicted label matches our actual label.

In [ ]:
corrects = torch.eq(labels, pred_labels)

We can then loop through all of the examples over our model's predictions and store all the examples the model got incorrect into an array.

Then, we sort these incorrect examples by how confident they were, with the most confident being first.

In [ ]:
incorrect_examples = []

for image, label, prob, correct in zip(images, labels, probs, corrects):
    if not correct:
        incorrect_examples.append((image, label, prob))

incorrect_examples.sort(reverse=True, key=lambda x: torch.max(x[2], dim=0).values)

We can then plot the incorrectly predicted images along with how confident they were on the actual label and how confident they were at the incorrect label.

In [ ]:
def plot_most_incorrect(incorrect, n_images):

    rows = int(np.sqrt(n_images))
    cols = int(np.sqrt(n_images))

    fig = plt.figure(figsize=(20, 10))
    for i in range(rows*cols):
        ax = fig.add_subplot(rows, cols, i+1)
        image, true_label, probs = incorrect[i]
        true_prob = probs[true_label]
        incorrect_prob, incorrect_label = torch.max(probs, dim=0)
        ax.imshow(image.view(28, 28).cpu().numpy(), cmap='bone')
        ax.set_title(f'true label: {true_label} ({true_prob:.3f})\n'
                     f'pred label: {incorrect_label} ({incorrect_prob:.3f})')
        ax.axis('off')
    fig.subplots_adjust(hspace=0.5)

Below we can see the 25 images the model got incorrect and was most confident about. A lot of these digits are irregular, so it's difficult for the model to do well on these. The images that do look fine, if you squint you can sort of see why the model got it wrong.

In [ ]:
N_IMAGES = 25

plot_most_incorrect(incorrect_examples, N_IMAGES)

### Summary

In this notebook we have shown: 

- Loading Torchvision datasets,
- Loading transforms to augment and normalize our data,
- Defining a MLP,
- Training a model to achieve > 98% accuracy,
- Viewing our mistakes of model.

## Clear the Memory

Before moving on, please execute the following cell to clear up the CPU memory. This is required to move on to the next notebook.

In [ ]:
#import IPython
#app = IPython.Application.instance()
#app.kernel.do_shutdown(True)

## Next

In this section you learned how to apply the Multilayer Perceptron for data regression. In the notebook [_03-multilayer-percepetron-regression.ipynb_](03-multilayer-percepetron-regression.ipynb).